# Suspicious Kerberos RC4 Ticket Encryption
Detects service ticket requests using RC4 encryption type

## Rule Content
```
- title: Suspicious Kerberos RC4 Ticket Encryption
  id: 496a0e47-0a33-4dca-b009-9e6ca3591f39
  status: experimental
  references:
  - https://adsecurity.org/?p=3458
  - https://www.trimarcsecurity.com/single-post/TrimarcResearch/Detecting-Kerberoasting-Activity
  tags:
  - attack.credential_access
  - attack.t1208
  description: Detects service ticket requests using RC4 encryption type
  logsource:
    product: windows
    service: security
    category: null
  detection:
    selection:
      EventID: 4769
      TicketOptions: '0x40810000'
      TicketEncryptionType: '0x17'
    reduction:
    - ServiceName: $*
    condition: selection and not reduction
  falsepositives:
  - Service accounts used on legacy systems (e.g. NetApp)
  - Windows Domains with DFL 2003 and legacy systems
  level: medium

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-endpoint-winevent-security-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='((event_id:"4769" AND ticket_options:"0x40810000" AND ticket_encryption_type:"0x17") AND (NOT (service_name.keyword:$*)))')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()